In [1]:
from pyanitools import anidataloader
import h5py

'''
# load in the entire dataset, this is just one of them
data = []

# TODO: CHANGE THIS BACK TO 9 WHEN WE'RE DONE
for i in range(1,3):
    data = data + list(anidataloader('data/ANI-1_release/ani_gdb_s0' + str(i) + '.h5'))
'''

"\n# load in the entire dataset, this is just one of them\ndata = []\n\n# TODO: CHANGE THIS BACK TO 9 WHEN WE'RE DONE\nfor i in range(1,3):\n    data = data + list(anidataloader('data/ANI-1_release/ani_gdb_s0' + str(i) + '.h5'))\n"

In [2]:
data=anidataloader("../ANI-1_release/ani_gdb_s04.h5")

/mnt/c/Users/Camille/msse/CHEM277B/final_project/ANI_1/pyanitools.py:47: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.store = h5py.File(store_file)


In [35]:
data=anidataloader("./checkpoint.pt")

/mnt/c/Users/Camille/msse/CHEM277B/final_project/ANI_1/pyanitools.py:47: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.store = h5py.File(store_file)


OSError: Unable to create file (unable to open file: name = './checkpoint.pt', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)

In [36]:
data_list = [item for item in data]

In [4]:
import numpy as np


def calc_f_C(Rij, RC):
    f_C_value = 0.5 * np.cos(np.pi * Rij / RC) + 0.5
    indicator = ((Rij <= RC) & (Rij != 0)).astype(float) # Make f_C(0)=0 to make sure the sum in distance conversion function and radial conversion function can run with j=i
    return f_C_value * indicator

def radial_component(Rijs, eta, Rs, RC=5.2):
    # Rijs is a 1d array, all other parameters are scalars
    f_C_values = calc_f_C(Rijs, RC)
    individual_components = np.exp(-eta * (Rijs - Rs) ** 2) * f_C_values
    return np.sum(individual_components)

def angular_component(Rij_vectors, Rik_vectors, zeta, theta_s, eta, Rs, RC=3.5):
    # Rij_vectors and Rik_vectors are 2d arrays with shape (n_atoms, 3), all other parameters are scalars
    # calculate theta_ijk values from vector operations
    dot_products = Rij_vectors.dot(Rik_vectors.T)
    Rij_norms = np.linalg.norm(Rij_vectors, axis=-1)
    Rik_norms = np.linalg.norm(Rik_vectors, axis=-1)
    norms = Rij_norms.reshape((-1, 1)).dot(Rik_norms.reshape((1, -1)))
    cos_values = np.clip(dot_products / (norms + 1e-8), -1, 1)
    theta_ijks = np.arccos(cos_values)
    theta_ijk_filter = (theta_ijks != 0).astype(float)
    mean_dists = (Rij_norms.reshape((-1, 1)) + Rik_norms.reshape((1, -1))) / 2
    f_C_values_Rij = calc_f_C(Rij_norms, RC)
    f_C_values_Rik = calc_f_C(Rik_norms, RC)
    f_C_values = f_C_values_Rij.reshape((-1, 1)).dot(f_C_values_Rik.reshape((1, -1)))
    individual_components = (1 + np.cos(theta_ijks - theta_s)) ** zeta * np.exp(-eta * (mean_dists - Rs) ** 2) * f_C_values * theta_ijk_filter
    return 2 ** (1 - zeta) * np.sum(individual_components)

def calc_aev(atom_types, coords, i_index):
    relative_coordinates = coords - coords[i_index]
    nearby_atom_indicator = np.linalg.norm(relative_coordinates, axis=-1) < 5.3
    relative_coordinates = relative_coordinates[nearby_atom_indicator]
    atom_types = atom_types[nearby_atom_indicator]
    radial_aev = np.array([radial_component(np.linalg.norm(relative_coordinates[atom_types == atom], axis=-1), eta, Rs) \
                           for atom in [0, 1, 2, 3] for eta in [16] \
                           for Rs in [0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,\
                                   3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250]])
    angular_aev = np.array([angular_component(relative_coordinates[atom_types == atom_j], relative_coordinates[atom_types == atom_k],\
                                             zeta, theta_s, eta, Rs) \
                            for atom_j in [0, 1, 2, 3] for atom_k in range(atom_j, 4) for zeta in [32] \
                            for theta_s in [0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243]\
                            for eta in [8] for Rs in [0.900000,1.550000,2.200000,2.850000]])
    return np.concatenate([radial_aev, angular_aev])

In [5]:
import torch
from torch import nn

class Gaussian(nn.Module):
    def forward(self, x):
        return torch.exp(- x * x)

class ANI(nn.Module):
    def __init__(self, architecture_dict):
        super(ANI, self).__init__()
        self.subnets = nn.ModuleDict({'C':ANI_sub(architecture_dict['C']), 
                                      'H':ANI_sub(architecture_dict['H']),
                                      'O':ANI_sub(architecture_dict['O']),
                                      'N':ANI_sub(architecture_dict['N'])})
        
    def forward(self, aev_inputs, atom_types):
        total_energy = torch.tensor(0)
        for atom in ["H", "C", "O", "N"]:
            if atom in atom_types:
                atom_specific_aev = torch.tensor([aev_inputs[idx] for idx in range(len(atom_types)) if atom_types[idx] == atom], dtype=torch.float)
                atomic_energies = self.subnets[atom](atom_specific_aev)
                #print("atom: " + atom + ", energies: " + str(atomic_energies.detach().numpy()))
                total_energy = torch.add(total_energy,torch.sum(atomic_energies))
        return total_energy
    
class ANI_sub(nn.Module):
    def __init__(self, architecture):
        super(ANI_sub, self).__init__()
            
        self.layers = architecture
        
        for m in self.layers:
            if type(m) == nn.Linear:
                d = m.weight.shape[1] #number of input nodes
                nn.init.uniform_(m.weight, a=-1/d, b=1/d) #initialize weights between -1/d and 1/d
                nn.init.zeros_(m.bias) #initialize biases to 0
    
    def forward(self, x):
        return self.layers(x)

In [6]:
def calc_loss(E_pred, E_target, tau):
    square_diff = torch.sum((E_pred - E_target) ** 2)
    return (tau * torch.exp(1/tau * square_diff))

In [7]:
arch_dict = {'C': nn.Sequential(
                                nn.Linear(384, 128),
                                Gaussian(),
                                nn.Linear(128,128),
                                Gaussian(),
                                nn.Linear(128,64),
                                Gaussian(),
                                nn.Linear(64,1)), 
              'H': nn.Sequential(
                                nn.Linear(384, 128),
                                Gaussian(),
                                nn.Linear(128,128),
                                Gaussian(),
                                nn.Linear(128,64),
                                Gaussian(),
                                nn.Linear(64,1)),
              'O': nn.Sequential(
                                nn.Linear(384, 128),
                                Gaussian(),
                                nn.Linear(128,128),
                                Gaussian(),
                                nn.Linear(128,64),
                                Gaussian(),
                                nn.Linear(64,1)),
              'N': nn.Sequential(
                                nn.Linear(384, 128),
                                Gaussian(),
                                nn.Linear(128,128),
                                Gaussian(),
                                nn.Linear(128,64),
                                Gaussian(),
                                nn.Linear(64,1))}

In [8]:
arch_dict = {'C': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.ReLU(),
                                nn.Linear(128,128),
                                nn.ReLU(),
                                nn.Linear(128,64),
                                nn.ReLU(),
                                nn.Linear(64,1)), 
              'H': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.ReLU(),
                                nn.Linear(128,128),
                                nn.ReLU(),
                                nn.Linear(128,64),
                                nn.ReLU(),
                                nn.Linear(64,1)),
              'O': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.ReLU(),
                                nn.Linear(128,128),
                                nn.ReLU(),
                                nn.Linear(128,64),
                                nn.ReLU(),
                                nn.Linear(64,1)),
              'N': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.ReLU(),
                                nn.Linear(128,128),
                                nn.ReLU(),
                                nn.Linear(128,64),
                                nn.ReLU(),
                                nn.Linear(64,1))}

In [9]:
arch_dict = {'C': nn.Sequential(
                                nn.Linear(384, 144),
                                nn.CELU(),
                                nn.Linear(144,112),
                                nn.CELU(),
                                nn.Linear(112,96),
                                nn.CELU(),
                                nn.Linear(96,1)), 
              'H': nn.Sequential(
                                nn.Linear(384, 160),
                                nn.CELU(),
                                nn.Linear(160,128),
                                nn.CELU(),
                                nn.Linear(128,96),
                                nn.CELU(),
                                nn.Linear(96,1)),
              'O': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.CELU(),
                                nn.Linear(128,112),
                                nn.CELU(),
                                nn.Linear(112,96),
                                nn.CELU(),
                                nn.Linear(96,1)),
              'N': nn.Sequential(
                                nn.Linear(384, 128),
                                nn.CELU(),
                                nn.Linear(128,112),
                                nn.CELU(),
                                nn.Linear(112,96),
                                nn.CELU(),
                                nn.Linear(96,1))}

In [10]:
model = ANI(arch_dict)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
arch_dict["C"].state_dict()

OrderedDict([('0.weight',
              tensor([[ 1.2885e-03, -2.3436e-03,  1.4488e-03,  ...,  8.8773e-04,
                        2.5624e-03,  1.5249e-03],
                      [ 9.0340e-04, -1.1479e-03, -1.9594e-03,  ...,  1.4659e-03,
                        1.1768e-03, -7.8144e-04],
                      [-1.7187e-03,  2.1308e-03, -1.5684e-04,  ..., -1.1917e-03,
                       -7.3699e-05, -9.3318e-04],
                      ...,
                      [ 2.1808e-03, -1.0092e-03, -3.0536e-04,  ...,  5.5375e-04,
                       -1.6926e-03,  3.1487e-04],
                      [-2.5882e-03, -1.6554e-03, -6.2413e-04,  ...,  1.1169e-03,
                        2.2570e-03,  8.3050e-04],
                      [ 6.6389e-04,  8.4671e-05, -1.0881e-03,  ...,  1.6101e-03,
                        1.9252e-03,  1.4508e-03]])),
             ('0.bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                

In [12]:
model.state_dict()

OrderedDict([('subnets.C.layers.0.weight',
              tensor([[ 1.2885e-03, -2.3436e-03,  1.4488e-03,  ...,  8.8773e-04,
                        2.5624e-03,  1.5249e-03],
                      [ 9.0340e-04, -1.1479e-03, -1.9594e-03,  ...,  1.4659e-03,
                        1.1768e-03, -7.8144e-04],
                      [-1.7187e-03,  2.1308e-03, -1.5684e-04,  ..., -1.1917e-03,
                       -7.3699e-05, -9.3318e-04],
                      ...,
                      [ 2.1808e-03, -1.0092e-03, -3.0536e-04,  ...,  5.5375e-04,
                       -1.6926e-03,  3.1487e-04],
                      [-2.5882e-03, -1.6554e-03, -6.2413e-04,  ...,  1.1169e-03,
                        2.2570e-03,  8.3050e-04],
                      [ 6.6389e-04,  8.4671e-05, -1.0881e-03,  ...,  1.6101e-03,
                        1.9252e-03,  1.4508e-03]])),
             ('subnets.C.layers.0.bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [13]:
import random
import os
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

In [14]:
data = [data_list[0]]
print(data[0]["energies"])
data[0]["energies"] = (np.array(data[0]["energies"]) - np.mean(data[0]["energies"])) / np.std(data[0]["energies"])
print(data[0]["energies"])

[-174.39633094 -174.40525352 -174.40431969 ... -174.38644692 -174.41533629
 -174.33884549]
[ 0.96374876  0.47277708  0.52416144 ...  1.50762523 -0.08203582
  4.12693187]


In [15]:
data = data_list[:10]

In [16]:
def train(data, arch_dict, checkpoint_file='', max_epochs=100, validation_size=0.2, batch_size=1024, use_gpu=False):
    tensorboard = SummaryWriter(flush_secs=1, max_queue=1)

    #Standardize energies to mean 0, std 1
    for mol in data:
        mol["energies"] = (np.array(mol["energies"]) - np.mean(mol["energies"])) / np.std(mol["energies"])
        
    #Split training and validation by molecules
    train_data, val_data = train_test_split(data, test_size=validation_size)
    
    #Initialize NN
    model = ANI(arch_dict)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    start_epoch = 0
    
    if os.path.isfile(checkpoint_file):
        checkpoint = torch.load(checkpoint_file)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        start_epoch = checkpoint['epoch']
        
    if use_gpu:
            model = model.to("cuda:0")
    
    index_molecules = range(len(train_data))
    total_conformations = np.sum([train_data[i]["coordinates"].shape[0] for i in range(len(train_data))])
    mapping={"H":0, "C":1, "N":2, "O":3}
    
    #Store epoch losses and validation losses
    epoch_losses = []
    val_losses = []
    for n_epoch in range(start_epoch, max_epochs):
        counter = 0
        epoch_loss = 0
        while counter <= 1:
        #while counter <= np.floor(total_conformations / batch_size): #(sum of the number of conformations for all the molecules) / batch_size => one epoch
            current_mol_idx = random.choice(index_molecules)
            current_molecule = train_data[current_mol_idx]
            all_conformation_idx = list(range(len(current_molecule["coordinates"])))
            random.shuffle(all_conformation_idx)
            current_batch_idx = all_conformation_idx[:batch_size]
            batch_conformation = current_molecule["coordinates"][current_batch_idx] #batch_size, N_a, 3
            batch_energies = current_molecule["energies"][current_batch_idx] #batch_size
            pred_energies = []
            square_diff = torch.tensor(0)
            optimizer.zero_grad()
            for i in range(batch_size // 10):
                conformation = batch_conformation[i] #N_a, 3
                elements=np.array([mapping[atom] for atom in current_molecule["species"]])
                aevs = [calc_aev(elements, conformation, a) for a in range(len(conformation))] #N_a, 384
                total_energy = model.forward(aevs, current_molecule["species"])
                square_diff = torch.add(square_diff, (total_energy - batch_energies[i]) **2)
            loss = torch.mean(square_diff)
            epoch_loss += loss.detach().cpu().item()
            loss.backward()
            optimizer.step()
            counter += 1
            print("Epoch %d/%d - Batch %d: Loss value - %.4f" % (n_epoch + 1, max_epochs, counter, loss.item()))
        epoch_loss = epoch_loss / counter #Take average
        epoch_losses.append(epoch_loss)
        
        #Validate
        val_square_diff = torch.tensor(0)
        with torch.no_grad():
            for mol in [val_data[0]]:
                pred_energies = []
                for conformation in mol["coordinates"][:100]:
                    elements=np.array([mapping[atom] for atom in mol["species"]])
                    aevs = [calc_aev(elements, conformation, a) for a in range(len(conformation))] #N_a, 384
                    total_energy = model.forward(aevs, mol["species"])
                    pred_energies.append(total_energy.detach().cpu().item())
                val_square_diff = torch.add(val_square_diff, np.sum((np.array(pred_energies) - mol["energies"][:100]) **2))
        
        val_loss = torch.mean(val_square_diff)
        val_losses.append(val_loss.detach().cpu().item())
        
        tensorboard.add_scalar("epoch_losses", epoch_loss, n_epoch + 1)
        tensorboard.add_scalar("val_losses", val_loss, n_epoch + 1)
        
        print("Epoch %d/%d - Loss: %.3f - Val loss: %.3f" % (n_epoch + 1, max_epochs, epoch_loss, val_loss))

        torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch' : n_epoch + 1
        }, checkpoint_file)
    return epoch_losses, val_losses

In [29]:
def predict_energies(test_data, arch_dict, checkpoint_file='', use_gpu=False):
    
    #Standardize energies to mean 0, std 1
    for mol in test_data:
        mol["energies"] = (np.array(mol["energies"]) - np.mean(mol["energies"])) / np.std(mol["energies"])
    
    #Initialize NN
    model = ANI(arch_dict)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    if os.path.isfile(checkpoint_file):
        checkpoint = torch.load(checkpoint_file)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        
    if use_gpu:
            model = model.to("cuda:0")
            
    mapping={"H":0, "C":1, "N":2, "O":3}
    
    test_losses = []
    mol_energies = {}
    with torch.no_grad():
        for i in range(len(test_data)):#mol in [test_data[0]]:
            pred_energies = []
            test_square_diff = torch.tensor(0)
            for conformation in test_data[i]["coordinates"][:100]:
                elements=np.array([mapping[atom] for atom in test_data[i]["species"]])
                aevs = [calc_aev(elements, conformation, a) for a in range(len(conformation))] #N_a, 384
                total_energy = model.forward(aevs, test_data[i]["species"])
                pred_energies.append(total_energy.detach().cpu().item())
            test_square_diff = torch.add(test_square_diff, np.sum((np.array(pred_energies) - test_data[i]["energies"][:100]) **2))
            
            test_loss = torch.mean(test_square_diff)
            test_losses.append(test_loss.detach().cpu().item())
            mol_energies[i] = pred_energies
        
    return {'Test Losses' : test_losses, 'Molecular Energies' : mol_energies}
    
    
    

In [18]:
epoch_losses, val_losses = train(data, arch_dict, checkpoint_file="checkpoint_new.pt", max_epochs=2, use_gpu=False)

KeyboardInterrupt: 

In [31]:
test_results = predict_energies(data_list[11:13], arch_dict, checkpoint_file="checkpoint_new.pt", use_gpu=False)

In [32]:
test_results['Test Losses']

[92.74125671386719, 134.27101135253906]

In [33]:
test_results['Molecular Energies']

{0: [-0.009646549820899963,
  -0.009964261204004288,
  -0.009505705907940865,
  -0.00903763622045517,
  -0.00953536294400692,
  -0.009131774306297302,
  -0.010018954053521156,
  -0.008910723961889744,
  -0.008405819535255432,
  -0.009888071566820145,
  -0.008982466533780098,
  -0.009132014587521553,
  -0.009828690439462662,
  -0.010067879222333431,
  -0.010285014286637306,
  -0.009342004545032978,
  -0.010473720729351044,
  -0.00999627634882927,
  -0.010263698175549507,
  -0.010059494525194168,
  -0.00862688384950161,
  -0.009600036777555943,
  -0.008371259085834026,
  -0.00992327556014061,
  -0.009171989746391773,
  -0.00934803206473589,
  -0.009268416091799736,
  -0.010218081995844841,
  -0.009339096024632454,
  -0.009258519858121872,
  -0.010074103251099586,
  -0.009614167734980583,
  -0.00942298211157322,
  -0.009218654595315456,
  -0.010076174512505531,
  -0.0094361687079072,
  -0.009446113370358944,
  -0.008790252730250359,
  -0.009675494395196438,
  -0.009977019391953945,
  -0.0

In [ ]:
model = ANI(arch_dict)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

if os.path.isfile('checkpoint.pt'):
    checkpoint = torch.load('checkpoint.pt')
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
optimizer.state_dict()

In [ ]:
epoch_losses

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
plt.imshow(np.array(aevs[1]) - np.array(aevs[0]))

In [ ]:
class ANI_sub_test(nn.Module):
    def __init__(self, architecture):
        super(ANI_sub_test, self).__init__()
            
        self.layers = architecture
        
        '''
        for m in self.layers:
            if type(m) == nn.Linear:
                d = m.weight.shape[1] #number of input nodes
                nn.init.uniform_(m.weight, a=-1/d, b=1/d) #initialize weights between -1/d and 1/d
                nn.init.zeros_(m.bias) #initialize biases to 0
        '''
    
    def forward(self, x):
        return self.layers(x)

In [ ]:
test_model = ANI_sub_test(nn.Sequential(
                                nn.Linear(384, 128),
                                Gaussian(),
                                nn.Linear(128,128),
                                Gaussian(),
                                nn.Linear(128,64),
                                Gaussian(),
                                nn.Linear(64,1)))
atomic_energies = test_model(torch.tensor(aevs[0], dtype=torch.float))

In [ ]:
atomic_energies

In [ ]:
atomic_energies = test_model(torch.tensor(aevs[1], dtype=torch.float))

In [ ]:
atomic_energies